# Viewport

> Card stack viewport with 3-section CSS Grid layout, slot rendering,
> and OOB section updates.

In [ ]:
#| default_exp components.viewport

In [ ]:
#| export
from typing import Any, Callable, List, Optional

from fasthtml.common import Div, Script, Hidden

# DaisyUI utilities
from cjm_fasthtml_daisyui.utilities.semantic_colors import shadow_dui
from cjm_fasthtml_daisyui.utilities.border_radius import border_radius

# Tailwind utilities
from cjm_fasthtml_tailwind.utilities.effects import shadow
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_direction, justify, items, gap, grid_display
)
from cjm_fasthtml_tailwind.utilities.layout import overflow, position, inset, z
from cjm_fasthtml_tailwind.utilities.interactivity import cursor, touch
from cjm_fasthtml_tailwind.utilities.sizing import w, h
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.core.base import combine_classes

# Local imports
from cjm_fasthtml_card_stack.core.config import CardStackConfig
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.models import CardStackState, CardRenderContext, CardStackUrls
from cjm_fasthtml_card_stack.core.constants import CardRole
from cjm_fasthtml_card_stack.helpers.focus import resolve_focus_slot, calculate_viewport_window
from cjm_fasthtml_card_stack.components.states import render_placeholder_card

## Mode Sync Script

Synchronizes the keyboard navigation mode with the rendered UI state after
HTMX swaps.

In [ ]:
#| export
def _render_mode_sync_script(
    active_mode: Optional[str] = None,  # Active keyboard mode name (None = navigation)
) -> Any:  # Script element that syncs keyboard mode state
    """Generate script to sync keyboard navigation mode with rendered UI state."""
    target_mode = active_mode if active_mode else "navigation"

    return Script(f"""
        (function() {{
            if (typeof window.kbNav !== 'undefined') {{
                const state = window.kbNav.getState();
                const currentMode = state ? state.currentMode : 'navigation';
                const targetMode = '{target_mode}';
                if (targetMode !== 'navigation' && currentMode !== targetMode) {{
                    window.kbNav.enterMode(targetMode);
                }} else if (targetMode === 'navigation' && currentMode !== 'navigation') {{
                    window.kbNav.exitMode();
                }}
            }}
        }})();
    """)

## Click-to-Focus Overlay

Transparent overlay added to context card slots when `click_to_focus` is enabled.
Intercepts all clicks, enforcing the navigate-first-then-interact pattern.

In [ ]:
#| export
def _render_click_overlay(
    item_index: int,  # Index of the item this slot represents
    urls: CardStackUrls,  # URL bundle for navigation
) -> Any:  # Transparent click overlay element
    """Render transparent click-to-focus overlay for a context card slot."""
    return Div(
        cls=combine_classes(
            position.absolute, inset(0), z(10),
            cursor.pointer
        ),
        hx_post=urls.nav_to_index,
        hx_vals=f'{{"target_index": {item_index}}}',
        hx_swap="none"
    )

## render_slot_card

Renders a single card for a viewport slot. Handles focused/context styling,
placeholder rendering, click-to-focus overlays, and `CardRenderContext` construction.

In [ ]:
#| export
def render_slot_card(
    slot_index: int,  # Index of this slot in the viewport (0-based)
    focus_slot: int,  # Which slot is the focused position
    card_items: List[Any],  # Full items list
    item_index: Optional[int],  # Item index for this slot (None for placeholder)
    render_card: Callable,  # Callback: (item, CardRenderContext) -> FT
    state: CardStackState,  # Current card stack state
    config: CardStackConfig,  # Card stack configuration
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    urls: CardStackUrls,  # URL bundle for navigation
    oob: bool = False,  # Whether to render as OOB swap
) -> Any:  # Slot content wrapper
    """Render a single card for a viewport slot."""
    slot_id = ids.viewport_slot(slot_index)
    is_focused = slot_index == focus_slot
    card_role: CardRole = "focused" if is_focused else "context"
    total_items = len(card_items)
    distance = slot_index - focus_slot

    # Determine placeholder type based on position relative to focus
    placeholder_type = "start" if slot_index < focus_slot else "end"

    # Render content
    if item_index is None:
        content = render_placeholder_card(placeholder_type)
    else:
        context = CardRenderContext(
            card_role=card_role,
            index=item_index,
            total_items=total_items,
            is_first=(item_index == 0),
            is_last=(item_index == total_items - 1),
            active_mode=state.active_mode,
            card_scale=state.card_scale,
            distance_from_focus=distance,
        )
        content = render_card(card_items[item_index], context)

    # Focus shadow styling for focused slot
    focus_cls = combine_classes(
        shadow.lg, shadow_dui.primary, border_radius.box
    ) if is_focused else ""

    # Mode sync script in focused slot OOB updates
    mode_sync = _render_mode_sync_script(state.active_mode) if (oob and is_focused) else None

    # Click-to-focus overlay for context cards
    click_overlay = None
    if config.click_to_focus and not is_focused and item_index is not None:
        click_overlay = _render_click_overlay(item_index, urls)

    # Slot container
    slot_cls = combine_classes(
        "viewport-slot",
        p(1) if not is_focused else "",
        w.full,
        focus_cls,
        position.relative if click_overlay else ""
    )

    return Div(
        content,
        click_overlay,
        mode_sync,
        id=slot_id,
        cls=slot_cls,
        tabindex="0" if is_focused else "-1",
        hx_swap_oob="innerHTML" if oob else None
    )

In [ ]:
# Test render_slot_card
from fasthtml.common import to_xml, P as FP
from cjm_fasthtml_card_stack.core.config import _reset_prefix_counter

_reset_prefix_counter()
config = CardStackConfig(prefix="test")
ids = CardStackHtmlIds(prefix="test")
state = CardStackState()
urls = CardStackUrls(nav_to_index="/card-stack/nav_to_index")
items_list = ["Item A", "Item B", "Item C", "Item D", "Item E"]

def simple_render(item, ctx):
    return FP(f"{item} [{ctx.card_role}]")

# Test focused card (slot 1, center of 3)
card_el = render_slot_card(
    slot_index=1, focus_slot=1, card_items=items_list, item_index=2,
    render_card=simple_render, state=state, config=config, ids=ids, urls=urls
)
html = to_xml(card_el)
assert 'id="test-viewport-slot-1"' in html
assert 'tabindex="0"' in html  # Focused is tabbable
assert "Item C [focused]" in html
assert "shadow-lg" in html  # Focus shadow
print("Focused card test passed!")

Focused card test passed!


In [ ]:
# Test context card
card_el = render_slot_card(
    slot_index=0, focus_slot=1, card_items=items_list, item_index=1,
    render_card=simple_render, state=state, config=config, ids=ids, urls=urls
)
html = to_xml(card_el)
assert 'tabindex="-1"' in html  # Context not tabbable
assert "Item B [context]" in html
assert "shadow-lg" not in html  # No focus shadow
print("Context card test passed!")

Context card test passed!


In [ ]:
# Test placeholder card (None item_index)
card_el = render_slot_card(
    slot_index=0, focus_slot=1, card_items=items_list, item_index=None,
    render_card=simple_render, state=state, config=config, ids=ids, urls=urls
)
html = to_xml(card_el)
assert "Beginning" in html  # Before focus -> "start" placeholder
print("Placeholder card test passed!")

Placeholder card test passed!


In [ ]:
# Test click-to-focus overlay
click_config = CardStackConfig(prefix="click", click_to_focus=True)
click_ids = CardStackHtmlIds(prefix="click")
card_el = render_slot_card(
    slot_index=0, focus_slot=1, card_items=items_list, item_index=1,
    render_card=simple_render, state=state, config=click_config, ids=click_ids, urls=urls
)
html = to_xml(card_el)
assert 'hx-post="/card-stack/nav_to_index"' in html  # Overlay present
assert 'relative' in html  # Container has relative positioning
assert 'inset-0' in html  # Overlay covers full area

# Focused card should NOT have overlay even with click_to_focus=True
card_el = render_slot_card(
    slot_index=1, focus_slot=1, card_items=items_list, item_index=2,
    render_card=simple_render, state=state, config=click_config, ids=click_ids, urls=urls
)
html = to_xml(card_el)
assert 'hx-post' not in html  # No overlay on focused card
print("Click-to-focus overlay tests passed!")

Click-to-focus overlay tests passed!


In [ ]:
# Test CardRenderContext is correctly populated
captured_contexts = []

def capturing_render(item, ctx):
    captured_contexts.append(ctx)
    return FP(item)

captured_contexts.clear()
render_slot_card(
    slot_index=1, focus_slot=1, card_items=items_list, item_index=0,
    render_card=capturing_render, state=CardStackState(active_mode="edit", card_scale=75),
    config=config, ids=ids, urls=urls
)
ctx = captured_contexts[0]
assert ctx.card_role == "focused"
assert ctx.index == 0
assert ctx.total_items == 5
assert ctx.is_first == True
assert ctx.is_last == False
assert ctx.active_mode == "edit"
assert ctx.card_scale == 75
assert ctx.distance_from_focus == 0
print("CardRenderContext population test passed!")

CardRenderContext population test passed!


## render_all_slots_oob

Renders all viewport sections with OOB swap for granular updates.
Returns OOB elements for the 3-section layout.

In [ ]:
#| export
def render_all_slots_oob(
    card_items: List[Any],  # All data items
    state: CardStackState,  # Current card stack state
    config: CardStackConfig,  # Card stack configuration
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    urls: CardStackUrls,  # URL bundle for navigation
    render_card: Callable,  # Card renderer callback
) -> List[Any]:  # List of OOB elements (3 sections)
    """Render all viewport sections with OOB swap for granular updates."""
    total_items = len(card_items)
    focus_slot = resolve_focus_slot(state.focus_position, state.visible_count)

    viewport_indices = calculate_viewport_window(
        state.focused_index, total_items, state.visible_count, state.focus_position
    )

    before_cards = []
    focused_card = None
    after_cards = []

    for slot_index, item_index in enumerate(viewport_indices):
        card_el = render_slot_card(
            slot_index=slot_index, focus_slot=focus_slot,
            card_items=card_items, item_index=item_index,
            render_card=render_card, state=state,
            config=config, ids=ids, urls=urls, oob=False,
        )

        if slot_index < focus_slot:
            before_cards.append(card_el)
        elif slot_index == focus_slot:
            focused_card = card_el
        else:
            after_cards.append(card_el)

    # Section styling
    section_cls = lambda alignment: combine_classes(
        flex_display, flex_direction.col, alignment, items.center,
        w.full, gap(4), overflow.hidden
    )

    before_section = Div(
        *before_cards,
        id=ids.viewport_section_before,
        cls=section_cls(justify.end),
        hx_swap_oob="innerHTML"
    )

    mode_sync = _render_mode_sync_script(state.active_mode)
    focused_section = Div(
        focused_card, mode_sync,
        id=ids.viewport_section_focused,
        cls=combine_classes(flex_display, justify.center, items.center, w.full, p.x(2), p.b(4)),
        hx_swap_oob="innerHTML"
    )

    after_section = Div(
        *after_cards,
        id=ids.viewport_section_after,
        cls=section_cls(justify.start),
        hx_swap_oob="innerHTML"
    )

    return [before_section, focused_section, after_section]

In [ ]:
# Test render_all_slots_oob
state = CardStackState(focused_index=2, visible_count=3)
sections = render_all_slots_oob(
    card_items=items_list, state=state, config=config,
    ids=ids, urls=urls, render_card=simple_render
)
assert len(sections) == 3

html_before = to_xml(sections[0])
assert 'id="test-viewport-section-before"' in html_before
assert 'hx-swap-oob="innerHTML"' in html_before

html_focused = to_xml(sections[1])
assert 'id="test-viewport-section-focused"' in html_focused

html_after = to_xml(sections[2])
assert 'id="test-viewport-section-after"' in html_after
print("render_all_slots_oob tests passed!")

render_all_slots_oob tests passed!


## Grid Template Helpers

The CSS Grid template is determined by the focus position intent and stays
stable across visible_count changes:
- Center (None): `1fr auto 1fr`
- Bottom (-1): `1fr auto`
- Top (0): `auto 1fr`

In [ ]:
#| export
def _grid_template_rows(
    focus_position: Optional[int] = None,  # Focus slot offset (None=center, -1=bottom, 0=top)
) -> str:  # CSS grid-template-rows value
    """Compute CSS grid-template-rows based on focus position intent."""
    if focus_position is None:
        return "1fr auto 1fr"  # Center: always 3-section
    elif focus_position == 0:
        return "auto 1fr"  # Top: focused first
    elif focus_position < 0:
        return "1fr auto"  # Bottom: focused last
    else:
        return "1fr auto 1fr"  # Custom positive: always 3-section

In [ ]:
# Test grid template computation — stable across visible_count changes
assert _grid_template_rows(None) == "1fr auto 1fr"   # Center: always 3-section
assert _grid_template_rows(-1) == "1fr auto"          # Bottom: focused last
assert _grid_template_rows(0) == "auto 1fr"           # Top: focused first
assert _grid_template_rows(2) == "1fr auto 1fr"       # Custom positive: 3-section
print("Grid template tests passed!")

Grid template tests passed!


## render_viewport

Main viewport renderer. Builds the 3-section CSS Grid layout with
opacity reveal pattern.

In [ ]:
#| export
def render_viewport(
    card_items: List[Any],  # All data items
    state: CardStackState,  # Current card stack state
    config: CardStackConfig,  # Card stack configuration
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    urls: CardStackUrls,  # URL bundle for navigation
    render_card: Callable,  # Card renderer callback
    form_input_name: str = "focused_index",  # Name for the focused index hidden input
) -> Any:  # Viewport component with 3-section layout
    """Render the card stack viewport with 3-section CSS Grid layout."""
    total_items = len(card_items)
    focus_slot = resolve_focus_slot(state.focus_position, state.visible_count)

    viewport_indices = calculate_viewport_window(
        state.focused_index, total_items, state.visible_count, state.focus_position
    )

    before_cards = []
    focused_card = None
    after_cards = []

    for slot_index, item_index in enumerate(viewport_indices):
        card_el = render_slot_card(
            slot_index=slot_index, focus_slot=focus_slot,
            card_items=card_items, item_index=item_index,
            render_card=render_card, state=state,
            config=config, ids=ids, urls=urls, oob=False,
        )

        if slot_index < focus_slot:
            before_cards.append(card_el)
        elif slot_index == focus_slot:
            focused_card = card_el
        else:
            after_cards.append(card_el)

    # Section styling helper
    section_cls = lambda alignment: combine_classes(
        flex_display, flex_direction.col, alignment, items.center,
        w.full, gap(4), overflow.hidden
    )

    before_section = Div(
        *before_cards,
        id=ids.viewport_section_before,
        cls=section_cls(justify.end)
    )

    focused_section = Div(
        focused_card,
        id=ids.viewport_section_focused,
        cls=combine_classes(flex_display, justify.center, items.center, w.full, p.x(2), p.b(4))
    )

    after_section = Div(
        *after_cards,
        id=ids.viewport_section_after,
        cls=section_cls(justify.start)
    )

    # Grid template based on focus position intent (stable across count changes)
    grid_rows = _grid_template_rows(state.focus_position)

    inner_cls = combine_classes(grid_display, w.full, h.full, m.x.auto, gap(4))
    inner_style = f"grid-template-rows: {grid_rows}; max-width: {state.card_width}rem"

    outer_cls = combine_classes(w.full, p.x(2), p.y(2), overflow.hidden, touch.none)

    # Hidden input for focused index (needed for keyboard nav hx-include and OOB updates)
    focused_input = Hidden(
        id=ids.focused_index_input,
        name=form_input_name,
        value=str(state.focused_index),
    )

    return Div(
        Div(
            before_section,
            focused_section,
            after_section,
            id=ids.card_stack_inner,
            cls=inner_cls,
            style=inner_style
        ),
        _render_mode_sync_script(state.active_mode),
        focused_input,
        id=ids.card_stack,
        cls=outer_cls,
        style="opacity: 0; transition: opacity 150ms ease-in",
        data_focused_index=str(state.focused_index),
        data_total_items=str(total_items),
        data_visible_count=str(state.visible_count)
    )

In [ ]:
# Test render_viewport
state = CardStackState(focused_index=2, visible_count=5, card_width=60)
viewport = render_viewport(
    card_items=items_list, state=state, config=config,
    ids=ids, urls=urls, render_card=simple_render
)
html = to_xml(viewport)
assert 'id="test-card-stack"' in html
assert 'id="test-card-stack-inner"' in html
assert 'id="test-viewport-section-before"' in html
assert 'id="test-viewport-section-focused"' in html
assert 'id="test-viewport-section-after"' in html
assert 'opacity: 0' in html  # Opacity reveal pattern
assert 'data-focused-index="2"' in html
assert 'data-total-items="5"' in html
assert 'data-visible-count="5"' in html
assert '1fr auto 1fr' in html  # Center focus grid
assert 'max-width: 60rem' in html
assert 'touch-none' in html  # Touch gesture capture

# Verify focused_index hidden input is included
assert 'id="test-focused-index"' in html
assert 'name="focused_index"' in html
assert 'value="2"' in html
print("render_viewport tests passed!")

render_viewport tests passed!


In [ ]:
# Test bottom-anchored viewport
state = CardStackState(focused_index=2, visible_count=3, focus_position=-1)
viewport = render_viewport(
    card_items=items_list, state=state, config=config,
    ids=ids, urls=urls, render_card=simple_render
)
html = to_xml(viewport)
assert 'grid-template-rows: 1fr auto;' in html  # Bottom focus grid
assert 'Item C [focused]' in html  # Item at index 2 is focused
print("Bottom-anchored viewport test passed!")

Bottom-anchored viewport test passed!


In [ ]:
# Test viewport content correctness with center focus
state = CardStackState(focused_index=2, visible_count=3)
viewport = render_viewport(
    card_items=items_list, state=state, config=config,
    ids=ids, urls=urls, render_card=simple_render
)
html = to_xml(viewport)
assert "Item B [context]" in html   # Before focused
assert "Item C [focused]" in html   # Focused
assert "Item D [context]" in html   # After focused
assert "Item A" not in html          # Not visible
assert "Item E" not in html          # Not visible
print("Viewport content correctness test passed!")

Viewport content correctness test passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()